### Mini-Project: Your First Digit-Reading AI

**Goal:** Build, train, and test a complete Convolutional Neural Network (CNN) that can learn to recognize handwritten digits.

In [13]:
from torch.nn import Flatten
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, random_split, DataLoader
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, transforms


### Data Preprocessing and Splitting

This cell applies transformations to the dataset, including normalization and grayscale conversion, and splits the data into training, validation, and test sets. It also creates data loaders for efficient batch processing during training and evaluation.


In [14]:
transformer_recipe = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.1307], std=[0.3081]),
    transforms.Grayscale()
])
root_dir = datasets.ImageFolder(root='/Users/etsubfeleke/Documents/Pytorch_practice/MINST/minst_training', transform=transformer_recipe)
train_size = int(0.7 * len(root_dir))
val_size = int(0.1 * len(root_dir))
test_size = len(root_dir) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(root_dir, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [15]:
class CNNBlock(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(in_channels=1,out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(32 * 7 * 7,  128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    def forward (self, x):
        x = self.cnn(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.cnn2(x)
        x = self.relu2(x)
        x= self.max_pool2(x)
        x= x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x= self.fc2(x)
        return x

my_model = CNNBlock(num_classes=10)


ce_loss = nn.CrossEntropyLoss()
optimizer_sgd = optim.SGD(my_model.parameters(), lr=0.001)
batch_size = 64
num_epochs = 25
print(f"Layers: \n {my_model}")

Layers: 
 CNNBlock(
  (cnn): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


- **This script trains a neural network with early stopping to prevent overfitting.**  
- **It monitors training and validation loss each epoch to prevent overfitting.**  
- **Training stops automatically if validation loss doesn't improve for a set number of epochs (patience).**


**different epochs and batch sizes are used to see how they affect the model's performance.**
--- Epoch [20/20],  Batch: [656], Loss: 0.3238  
----  Test Accuracy: 96.381%  

--- Epoch [15/15],  Batch: [656], Loss: 0.0296  
----  Test Accuracy: 95.357%  

--- Epoch [10/10],  Batch: [656], Loss: 0.3713  
----  Test Accuracy: 92.881%  

--- Epoch [5/5],  Batch: [656], Loss: 0.1728  
----  Test Accuracy: 89.762%  


In [16]:

# Define the patience for early stopping
patience = 5
best_val_loss = float('inf')
epochs_no_improve = 0
num_epochs = 100  # Maximum number of epochs

for epoch in range(num_epochs):
    # Training loop
    my_model.train()
    train_loss = 0
    for images, labels in train_loader:
        outputs = my_model(images)
        loss = ce_loss(outputs, labels)
        
        optimizer_sgd.zero_grad()
        loss.backward()
        optimizer_sgd.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # Validation loop
    my_model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = my_model(images)
            loss = ce_loss(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    
    print(f"Epoch [{epoch+1}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    
    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

Epoch [1], Train Loss: 2.2586, Val Loss: 2.1920, Accuracy: 54.14%
Epoch [2], Train Loss: 2.0360, Val Loss: 1.7858, Accuracy: 71.38%
Epoch [3], Train Loss: 1.3054, Val Loss: 0.8610, Accuracy: 80.57%
Epoch [4], Train Loss: 0.6513, Val Loss: 0.5175, Accuracy: 86.29%
Epoch [5], Train Loss: 0.4544, Val Loss: 0.3955, Accuracy: 89.45%
Epoch [6], Train Loss: 0.3769, Val Loss: 0.3365, Accuracy: 90.71%
Epoch [7], Train Loss: 0.3329, Val Loss: 0.3037, Accuracy: 91.50%
Epoch [8], Train Loss: 0.3029, Val Loss: 0.2763, Accuracy: 92.05%
Epoch [9], Train Loss: 0.2798, Val Loss: 0.2583, Accuracy: 92.40%
Epoch [10], Train Loss: 0.2606, Val Loss: 0.2404, Accuracy: 93.02%
Epoch [11], Train Loss: 0.2446, Val Loss: 0.2245, Accuracy: 93.29%
Epoch [12], Train Loss: 0.2298, Val Loss: 0.2137, Accuracy: 93.88%
Epoch [13], Train Loss: 0.2171, Val Loss: 0.2015, Accuracy: 94.21%
Epoch [14], Train Loss: 0.2058, Val Loss: 0.1899, Accuracy: 94.48%
Epoch [15], Train Loss: 0.1954, Val Loss: 0.1824, Accuracy: 94.83%
Epoc

In [17]:
#test the model
print("Testing the model on test dataset...")  
my_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = my_model(images)
        loss = ce_loss(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.3f}%")
print("Testing completed.")
#plot the target and predicted values
import matplotlib.pyplot as plt
def plot_predictions(images, labels, predictions):
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for i in range(5):
        ax = axes[i]
        ax.imshow(images[i].squeeze(), cmap='gray')
        ax.set_title(f"Label: {labels[i]}, Pred: {predictions[i]}")
        ax.axis('off')
    plt.show()

Testing the model on test dataset...
Test Accuracy: 97.965%
Testing completed.
